# Jupyter Notebook for Validation of Results

## Overview

This notebook compares the results obtained from the two implementations, `shbundle` and `pyshbundle`, for a specific task.<br>
We will focus on evaluating the following:
1. Root Mean Squared Error (RMSE)
2. Normalized Root Mean Squared Error (NRMSE)
3. Difference in global area weighted water budget closure.
4. The difference in time series for a given basin.

## Evaluation Metrics

### 1. Root Mean Squared Error (RMSE)

The RMSE is a measure of the average magnitude of the errors between predicted and observed values.<br> It is calculated as follows:

$$ RMSE = \sqrt{\frac{1}{n}\Sigma_{i=1}^{n}{\Big({y_i - \bar{y}}\Big)^2}}$$

where:
- $n$ is the number of observations
- ${y_i}$ is the $i_{th}$ observation
- $\bar{y}$ is the mean of observations

### 2. Normalized Root Mean Squared Error (NRMSE)

NRMSE is a normalized version of RMSE, which provides a relative measure of the error compared to the range of the observed values. It is calculated as:

$$ NRMSE = \frac{RMSE}{\max(y) - \min(y)}$$

### 3. Difference in global area weighted water budget closure.

Since the total mass of Earth is conserved (at the accuracy we can currently observe), the mass change observed over any month should sum up to zero. To verify this, we area weight the solutions from `shbundle` & `pyshbundle` using the area_weighting() function by creating a global area grid. Then we multiply this with the global solutions at each time step and take the average, followed by plotting the results.

### 4. Difference in basin-average Time Series

Finally we create a basin-average TWSA time-series from both the implementations.

## Data Preparation

Before proceeding with the analysis, ensure that the data from both `shbundle` and `pyshbundle` are loaded into the notebook.

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import os

In [ ]:
''' Load tws processed data from shbundle as .mat file & from pyshbundle as xarray dataset '''
import scipy.io
# Load the .mat file
# data = scipy.io.loadmat('../pyshbundle/validation_data/tws_m.mat')
data = scipy.io.loadmat('/Users/vivek/Desktop/vivek_desktop/test/tws_m.mat')

# Access the variables in the .mat file
var1 = data['tws_m']

# temp=xr.open_dataset('../pyshbundle/validation_data/tws_py.nc', engine="netcdf4")
temp=xr.open_dataset('/Users/vivek/Desktop/vivek_desktop/test/ds_py_float64.nc', engine="netcdf4")

var2=temp.tws.values

#### Lets convert both datasets to a netcdf format for easier calculations.

* Converting `pyshbundle` processed data into netcdf format using xarray, to `ds_pysh`

In [ ]:
gs=1;
lon = np.arange(-180,180,gs)
lat = np.arange(89,-91,-gs)
ds_pysh = xr.Dataset(
    data_vars=dict(
        tws=(["time","lat", "lon"], var2)
    ),
    coords = {
        "time":(('time'),temp.time.data),
        "lat":lat,
        "lon":lon },
        attrs=dict(units="mm",),
)

* Same with `shubundle` processed data, to `ds_msh`

In [ ]:
ds_msh = xr.Dataset(
    data_vars=dict(
        tws=(["time","lat", "lon"], var1)
    ),
    coords = {
        "time":(('time'),temp.time.data),
        "lat":lat,
        "lon":lon },
    attrs=dict(units="mm",),
)

* Quick glance at the 2 datsets

In [ ]:
ds_pysh

In [ ]:
ds_msh

## 1. Gridwise RMSE calculation

Before finding the grid-wise RMSE values we need to ignore the data for the missing months.

In [ ]:
# Calculate the difference between the two datasets
diff = ds_msh['tws'].dropna(dim='time').values - ds_pysh['tws'].dropna(dim='time').values    # dropna is used to remove nan values, the dates where the GRACE data is missing

# Calculate the squared difference
squared_diff = diff**2

# Calculate the mean squared difference along the time axis
mean_squared_diff = np.mean(squared_diff, axis=0)

# Calculate the root mean squared error (RMSE)
gridwise_rmse = np.sqrt(mean_squared_diff)

In [ ]:
fig, (ax1) = plt.subplots(figsize=(6, 3), dpi=150, layout="constrained")    #gridspec_kw={'wspace': 0.01, 'hspace':0.2} 

# Set ticks and labels for the first subplot (ax1)
ax1.set_xticks(np.linspace(0, 359, 7)) 
ax1.set_xticklabels(['180W', '120W', '60W', '0', '60E', '120E', '180E'])
ax1.set_yticks(np.linspace(0, 179, 5))
ax1.set_yticklabels(['90N', '45N', '0', '45S', '90S'])

im1 = ax1.imshow(gridwise_rmse, cmap='jet')
cbar1 = fig.colorbar(im1, ax=ax1, pad=0.0, shrink=0.9)
ax1.set_title('RMSE between shbundle & pyshbundle generated TWS', fontsize=10, y=1.0)


plt.savefig('../pic/02_error_rmse.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
if np.all(gridwise_rmse < 1e-3):
    print('The two datasets are almost identical')

## 2. Gridwise NRMSE

In [ ]:
# Calculate the normalized root mean squared error (NRMSE)
gridwise_nrmse = gridwise_rmse/np.std(ds_msh['tws'].dropna(dim='time').values, axis=0)

In [ ]:
fig, (ax2) = plt.subplots(figsize=(6, 3), dpi=150, layout="constrained")    #gridspec_kw={'wspace': 0.01, 'hspace':0.2} 

# Set ticks and labels for the first subplot (ax2)
ax2.set_xticks(np.linspace(0, 359, 7))  # Adjust as needed
ax2.set_xticklabels(['180W', '120W', '60W', '0', '60E', '120E', '180E'])  # Adjust as needed
ax2.set_yticks(np.linspace(0, 179, 5))  # Adjust as needed
ax2.set_yticklabels(['90N', '45N', '0', '45S', '90S'])  # Adjust as needed

im1 = ax2.imshow(gridwise_nrmse, cmap='jet')
cbar1 = fig.colorbar(im1, ax=ax2, pad=0.0, shrink=0.9)
ax2.set_title('NRMSE between shbundle & pyshbundle generated TWS', fontsize=10, y=1)


plt.savefig('../pic/02_error_nrmse.png', dpi=300, bbox_inches='tight')
plt.show()

## 3. Global area weighted water budget closure

In [ ]:
# Area of grids
from pyshbundle.hydro import area_weighting
global_grid_area=area_weighting(1)
global_grid_area_sum = np.sum(global_grid_area)
print('global surface area in m\u00b2:', global_grid_area_sum)

* Calculate the global area weighted water budget closure error

In [ ]:
# Create a copy of the datasets
ds_msh_area_weighted, ds_pysh_area_weighted = ds_msh.copy(), ds_pysh.copy()

# Area weight with the global grid area and calculate the sum over lat and lon
ds_msh_area_weighted = ds_msh['tws']*global_grid_area / global_grid_area_sum
ds_msh_area_weighted = ds_msh_area_weighted.sum(dim=['lat', 'lon'])

# Same for the pyshbundle dataset
ds_pysh_area_weighted = ds_pysh['tws']*global_grid_area / global_grid_area_sum
ds_pysh_area_weighted = ds_pysh_area_weighted.sum(dim=['lat', 'lon'])

diff_global = ds_msh_area_weighted - ds_pysh_area_weighted

# Reinsert the NaN values where the GRACE data is missing and the time coordinate
ds_msh_area_weighted=ds_msh_area_weighted.where(~np.isnan(ds_pysh['tws'][:,0,0]), np.nan)
ds_pysh_area_weighted=ds_pysh_area_weighted.where(~np.isnan(ds_pysh['tws'][:,0,0]), np.nan)
diff_global=diff_global.where(~np.isnan(ds_pysh['tws'][:,0,0]), np.nan)

In [ ]:
# Create a figure with three subplots
fig, axes = plt.subplots(3, 1, figsize=(12, 15),)
fig.suptitle('Global water budget closure', fontsize=16)

# Plot 1
ds_msh_area_weighted.plot(ax=axes[0], color='#807dba', label='SHbundle')
axes[0].set_title('a) Water Budget Closure with SHbundle', fontsize=14)
axes[0].set_ylabel('Area Weighted TWSA (in cm)', fontsize=9)
axes[0].legend()
axes[0].grid(True, alpha=0.7)

# Plot 2
ds_pysh_area_weighted.plot(ax=axes[1], color='#4292c6', label='PySHbundle')
axes[1].set_title('b) Water Budget Closure with PySHbundle', fontsize=14)
axes[1].set_ylabel('Area Weighted TWSA (in cm)', fontsize=9)
axes[1].legend()
axes[1].grid(True, alpha=0.7)

# Plot 3
diff_global.plot(ax=axes[2], color='#525252', label='Difference')
axes[2].set_title('Difference between SHbundle and PySHbundle (a-b)', fontsize=14)
axes[2].set_ylabel('Area Weighted TWSA (in cm)', fontsize=9)
axes[2].legend()
axes[2].grid(True, alpha=0.7)

# Adjust layout for better spacing
plt.tight_layout()
plt.show()

## 4. Difference in basin-average Time Series

In [ ]:
import geopandas as gpd

In [ ]:
path_shapefile = '../pyshbundle/data/mrb_shapefiles/mrb_basins.shp'
shp = gpd.read_file(path_shapefile)
shp.plot(figsize=(8, 4))  

In [ ]:
basin_name='KRISHNA'
shp_basin=shp[shp['RIVER_BASI']==basin_name]
print(shp_basin.head(), '\n')
shp_basin.plot()
basin_area=np.float64(shp_basin['SUM_SUB_AR'].values[0])*1e6          # basin area already in m^2
print('Basin area is :', basin_area, 'm\u00b2');

In [ ]:
from pyshbundle.hydro import Basinaverage
_, basin_avg_tws_msh = Basinaverage(ds_msh, gs, shp_basin, basin_area)
_, basin_avg_tws_pysh = Basinaverage(ds_pysh, gs, shp_basin, basin_area)

In [ ]:
new_dates=pd.date_range(start=basin_avg_tws_msh.time[0].values, 
                        end=basin_avg_tws_msh.time[-1].values, freq='ME',)
new_dates

In [ ]:
# Empty dataset for the gapped data, msh
basin_avg_tws_gapped_msh = xr.Dataset(
        data_vars = dict(   tws=(["time"], np.nan*np.arange(len(new_dates)))),
        coords=dict(time=new_dates),);
# Empty dataset for the gapped data, pysh
basin_avg_tws_gapped_pysh = xr.Dataset(
        data_vars = dict(   tws=(["time"], np.nan*np.arange(len(new_dates)))),
        coords=dict(time=new_dates),);

In [ ]:
#
basin_avg_tws_gapped_msh['tws'] = basin_avg_tws_msh['tws'].where(
    basin_avg_tws_msh['time'].isin(basin_avg_tws_gapped_msh['time']),)
#
basin_avg_tws_gapped_pysh['tws'] = basin_avg_tws_pysh['tws'].where(
    basin_avg_tws_pysh['time'].isin(basin_avg_tws_gapped_pysh['time']),)

diff_global_basin = basin_avg_tws_gapped_msh['tws'] - basin_avg_tws_gapped_pysh['tws']

In [ ]:
# Create a figure with three subplots
fig, axes = plt.subplots(3, 1, figsize=(12, 15),)
fig.suptitle('Basin average comparision', fontsize=16)

# Plot 1
basin_avg_tws_gapped_msh['tws'].plot(ax=axes[0], color='#807dba', label='SHbundle')
axes[0].set_title('a) Basin average with SHbundle', fontsize=14)
axes[0].set_ylabel('TWSA (in cm)', fontsize=9)
axes[0].legend()
axes[0].grid(True, alpha=0.7)

# Plot 2
basin_avg_tws_gapped_pysh['tws'].plot(ax=axes[1], color='#4292c6', label='PySHbundle')
axes[1].set_title('b) Basin average with with PySHbundle', fontsize=14)
axes[1].set_ylabel('TWSA (in cm)', fontsize=9)
axes[1].legend()
axes[1].grid(True, alpha=0.7)

# Plot 3
diff_global_basin.plot(ax=axes[2], color='#525252', label='Difference')
axes[2].set_title('Difference between SHbundle and PySHbundle (a-b)', fontsize=14)
axes[2].set_ylabel('Area Weighted TWSA (in cm)', fontsize=9)
axes[2].legend()
axes[2].grid(True, alpha=0.7)

# Adjust layout for better spacing
plt.tight_layout()
plt.show()

* Thank you for reading!